In [1]:
!pip install -q gradio pymupdf sentence-transformers transformers torch

import gradio as gr
import fitz  # PyMuPDF
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.5 MB/s eta 0:00:00


In [2]:
# Configuración
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

EMBEDDING_MODEL = "all-mpnet-base-v2"
QA_MODEL = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [3]:
# Cargar modelos
modelo_emb = SentenceTransformer(EMBEDDING_MODEL).to(DEVICE)
tokenizer_qa = AutoTokenizer.from_pretrained(QA_MODEL)
modelo_qa = AutoModelForQuestionAnswering.from_pretrained(QA_MODEL).to(DEVICE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Variables globales
df_segmentos = None

def extraer_segmentos(pdf_bytes):
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    segmentos = []
    for pagina in doc:
        texto = pagina.get_text("text")
        for parrafo in texto.split('\n'):
            limpio = ' '.join(parrafo.strip().split())
            if len(limpio.split()) >= 15:
                max_words = 150
                words = limpio.split()
                for i in range(0, len(words), max_words):
                    segmento = ' '.join(words[i:i+max_words])
                    segmentos.append(segmento)
    return segmentos

def construir_indice(segmentos):
    embeddings = modelo_emb.encode(segmentos, normalize_embeddings=True)
    return pd.DataFrame({"texto": segmentos, "embeddings": list(embeddings)})

def recuperar_contexto(pregunta, df, k=3, umbral=0.7):
    emb_pregunta = modelo_emb.encode([pregunta], normalize_embeddings=True)[0]
    # Calcular similitud coseno manualmente
    import numpy as np
    sims = df['embeddings'].apply(lambda x: np.dot(x, emb_pregunta))
    df_top = df[sims >= umbral].copy()
    df_top['sim'] = sims[sims >= umbral]
    df_top = df_top.sort_values(by='sim', ascending=False).head(k)
    if df_top.empty:
        return None
    return df_top["texto"].tolist()

def responder_qa(pregunta, contextos):
    if not contextos:
        return "No tengo información suficiente para responder a eso."
    respuestas = []
    for ctx in contextos:
        inputs = tokenizer_qa.encode_plus(pregunta, ctx, return_tensors="pt").to(DEVICE)
        outputs = modelo_qa(**inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        answer = tokenizer_qa.convert_tokens_to_string(
            tokenizer_qa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
        )
        respuestas.append(answer)

    # Escoger la respuesta más larga no vacía (heurística simple)
    respuestas_validas = [r for r in respuestas if r.strip() != "" and r.strip() != "[CLS]" and r.strip() != "[SEP]"]
    if respuestas_validas:
        return max(respuestas_validas, key=len)
    else:
        return "No tengo información suficiente para responder a eso."

In [5]:
# --- Funciones interfaz Gradio ---

def cargar_pdf(pdf):
    global df_segmentos
    try:
        with open(pdf.name, "rb") as f:
            contenido = f.read()
        segmentos = extraer_segmentos(contenido)
        if not segmentos:
            return "Error: No se pudo extraer contenido útil del PDF.", ""
        df_segmentos = construir_indice(segmentos)
        return f"PDF procesado: {pdf.name.split('/')[-1]} ({len(segmentos)} segmentos)", ""
    except Exception as e:
        return f"Error al procesar PDF: {str(e)}", ""

def consultar(pregunta, k, umbral):
    if not pregunta.strip():
        return "Por favor ingresa una pregunta válida."
    contextos = recuperar_contexto(pregunta, df_segmentos, k, umbral/100)
    respuesta = responder_qa(pregunta, contextos)
    contexto_info = ""
    if contextos:
        contexto_info = "\n\nContexto usado:\n" + "\n".join([f"- {c[:100]}..." for c in contextos])
    return respuesta + contexto_info

In [6]:
with gr.Blocks() as demo:
    gr.Markdown("# Chatbot PDF con BERT QA extractiva\nCarga un PDF y pregunta sobre su contenido")
    with gr.Row():
        with gr.Column(scale=1):
            archivo_pdf = gr.File(label="Sube tu PDF", type="filepath")
            boton_cargar = gr.Button("Procesar PDF")
            estado = gr.Textbox(label="Estado", interactive=False)
        with gr.Column(scale=2):
            entrada = gr.Textbox(label="Tu pregunta")
            topk = gr.Slider(1,5,value=3,step=1,label="N° fragmentos a considerar")
            umbral = gr.Slider(50,95,value=70,step=5,label="Umbral similitud (%)")
            boton_preguntar = gr.Button("Preguntar")
            salida = gr.Textbox(label="Respuesta", interactive=False)
    boton_cargar.click(cargar_pdf, inputs=archivo_pdf, outputs=[estado, salida])
    boton_preguntar.click(consultar, inputs=[entrada, topk, umbral], outputs=salida)
    entrada.submit(consultar, inputs=[entrada, topk, umbral], outputs=salida)

In [7]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d0145bb029f4d48fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import pandas as pd
from datetime import datetime
import time

# === Evaluador de respuestas ===
def evaluar_respuesta(respuesta_obtenida, respuesta_esperada, tipo_pregunta):
    if tipo_pregunta == "fuera_dominio":
        if "no tengo información" in respuesta_obtenida.lower():
            return 5
        else:
            return 1
    else:
        respuesta_obtenida_clean = respuesta_obtenida.lower().strip()
        respuesta_esperada_clean = respuesta_esperada.lower().strip()

        if (respuesta_obtenida_clean == respuesta_esperada_clean or
            respuesta_esperada_clean in respuesta_obtenida_clean):
            return 5

        elementos_clave = [word for word in respuesta_esperada_clean.split() if len(word) > 3]

        if not elementos_clave:
            palabras_comunes = set(respuesta_obtenida_clean.split()) & set(respuesta_esperada_clean.split())
            return 3 if palabras_comunes else 1

        coincidencias = sum(1 for word in elementos_clave if word in respuesta_obtenida_clean)
        proporcion_coincidencias = coincidencias / len(elementos_clave)

        if proporcion_coincidencias >= 0.75:
            return 4
        elif proporcion_coincidencias >= 0.5:
            return 3
        elif proporcion_coincidencias >= 0.25:
            return 2
        else:
            return 1

# === Script de pruebas masivas ===
def pruebas_masivas(csv_entrada, csv_salida):
    df = pd.read_csv(csv_entrada)

    resultados = []
    for _, row in df.iterrows():
        tipo = row["tipo_pregunta"]
        pregunta = row["pregunta"]
        esperado = row["respuesta_esperada"]

        # Medir tiempo de respuesta
        inicio = time.time()
        obtenido = consultar(pregunta,3,70)  # <- tu función principal
        fin = time.time()
        tiempo_respuesta = fin - inicio

        puntaje = evaluar_respuesta(obtenido, esperado, tipo)

        resultados.append({
            "tipo_pregunta": tipo,
            "pregunta": pregunta,
            "respuesta_esperada": esperado,
            "respuesta_obtenida": obtenido,
            "puntaje": puntaje,
            "tiempo_seg": round(tiempo_respuesta, 3)
        })

    df_result = pd.DataFrame(resultados)

    # Métricas generales
    promedio_puntaje = df_result["puntaje"].mean()
    aciertos = (df_result["puntaje"] >= 4).sum()
    porcentaje_aciertos = (aciertos / len(df_result)) * 100
    promedio_tiempo = df_result["tiempo_seg"].mean()

    # Guardar resultados
    df_result.to_csv(csv_salida, index=False)
    print(f"Resultados guardados en {csv_salida}")
    print(f"Promedio de puntaje: {promedio_puntaje:.2f}")
    print(f"Aciertos (puntaje >= 4): {aciertos} / {len(df_result)} ({porcentaje_aciertos:.2f}%)")
    print(f"Tiempo promedio por respuesta: {promedio_tiempo:.3f} segundos")

In [13]:
# === Ejemplo de uso ===
pruebas_masivas(
    csv_entrada="/content/drive/MyDrive/Proyectos/Versiones ChatBot/2 - BERT/preguntas_prueba.csv",
    csv_salida=f"/content/drive/MyDrive/Proyectos/Versiones ChatBot/2 - BERT/resultados_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
)

Resultados guardados en /content/drive/MyDrive/Proyectos/Versiones ChatBot/2 - BERT/resultados_20250815_213840.csv
Promedio de puntaje: 1.98
Aciertos (puntaje >= 4): 21 / 102 (20.59%)
Tiempo promedio por respuesta: 1.400 segundos
